# **Image Classification with Convolutional Neural Networks (CNNs)**





# **Building a Convolutional Neural Network with Keras**

**Import Libraries**

In [1]:
import numpy as np
import pandas as pd

import psutil
import GPUtil

import os,sys,humanize,psutil,GPUtil

import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as mpimg
import imageio

%matplotlib inline

import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import ToTensor
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

**Memory Utilization Checkup (Before/After Model Execution)**

In [2]:
#CPU AND GPU LIBRARIES INSTALLATION (if you find error on import of psutil and GPUtil, please refer below)
!pip install psutil
!pip install GPUtil

In [3]:
#CPU MEMORY UTILIZATION

!cat /proc/meminfo

MemTotal:       16308040 kB
MemFree:         7672732 kB
MemAvailable:   10241100 kB
Buffers:          200976 kB
Cached:          2632248 kB
SwapCached:            0 kB
Active:          1168216 kB
Inactive:        6674948 kB
Active(anon):       2820 kB
Inactive(anon):  5150240 kB
Active(file):    1165396 kB
Inactive(file):  1524708 kB
Unevictable:          16 kB
Mlocked:              16 kB
SwapTotal:       2097148 kB
SwapFree:        2097148 kB
Dirty:              2148 kB
Writeback:             0 kB
AnonPages:       5009772 kB
Mapped:          1210652 kB
Shmem:            143120 kB
KReclaimable:     218720 kB
Slab:             371956 kB
SReclaimable:     218720 kB
SUnreclaim:       153236 kB
KernelStack:       19552 kB
PageTables:        58976 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    10251168 kB
Committed_AS:   19697428 kB
VmallocTotal:   34359738367 kB
VmallocUsed:      106388 kB
VmallocChunk:          0 kB
Percpu:          

In [4]:
!grep MemTotal /proc/meminfo

MemTotal:       16308040 kB


In [5]:
#GPU MEMORY UTILIZATION 
!nvidia-smi --query-gpu=memory.total --format=csv

memory.total [MiB]
11264 MiB


In [6]:
# Function
def mem_report():
  print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))
  
  GPUs = GPUtil.getGPUs()
  for i, gpu in enumerate(GPUs):
    print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))

mem_report()

CPU RAM Free: 10.5 GB
GPU 0 ... Mem Free: 8918MB / 11264MB | Utilization  19%


## **Let's Start with Model Implementation**

**Data Acquisition and ImageDataGenerator**


In [7]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

In [8]:
train_dir = os.path.join('/home/ines/code/ptb-xl/X_train/')
test_dir = os.path.join('/home/ines/code/ptb-xl/X_test/')

In [9]:
train_dataset = ImageFolder(root=train_dir, transform=transform)
test_dataset = ImageFolder(root=test_dir, transform=transform)

batch_size = 32
train_dl = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
test_dl = DataLoader(test_dataset, batch_size = batch_size, shuffle=True)

In [10]:
a = 0
for images, labels in train_dl:
    print(labels)
    a += 1
    if a == 5:
        break

tensor([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 0, 0])
tensor([0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 0, 1, 1, 1, 1, 1])
tensor([1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        0, 1, 0, 1, 0, 1, 0, 1])


**Building the Model**

In [11]:
a = 0
for images, labels in train_dl:
    print(images.shape)
    a += 1
    if a == 5:
        break

torch.Size([32, 3, 64, 64])
torch.Size([32, 3, 64, 64])
torch.Size([32, 3, 64, 64])
torch.Size([32, 3, 64, 64])
torch.Size([32, 3, 64, 64])


In [12]:
class cnn2d(nn.Module):
    def __init__(self):
        super(cnn2d, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=(3,3), bias=True)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=(3,3), bias=True)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=(3,3), bias=True)
        self.conv4 = nn.Conv2d(256, 64, kernel_size=(3,3), bias=True)

        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(0.25)
        self.maxpool2d = nn.MaxPool2d(2, 2)

        self.dense1 = nn.Linear(64*2*2, 8, bias=True)
        self.dense2 = nn.Linear(8, 1)

    def forward(self, x):
        x = self.maxpool2d(F.relu(self.conv1(x)))
        x = self.dropout(x)

        x = self.maxpool2d(F.relu(self.conv2(x)))

        x = self.maxpool2d(F.relu(self.conv3(x)))
        x = self.dropout(x)

        x = self.maxpool2d(F.relu(self.conv4(x)))
        x = self.dropout(x)

        x = self.flatten(x)

        x = F.relu(self.dense1(x))
        output = torch.sigmoid(self.dense2(x))

        return output


# 모델 인스턴스 생성
model = cnn2d()
print(model)

cnn2d(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(256, 64, kernel_size=(3, 3), stride=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dropout): Dropout(p=0.25, inplace=False)
  (maxpool2d): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dense1): Linear(in_features=256, out_features=8, bias=True)
  (dense2): Linear(in_features=8, out_features=1, bias=True)
)


In [13]:
'''
model = tf.keras.models.Sequential([
                                    
    # First convolution layer 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(64, 64, 3),use_bias=True),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),

    # Second convolution layer 
    tf.keras.layers.Conv2D(128, (3,3), activation='relu',use_bias=True),
    tf.keras.layers.MaxPooling2D(2,2),
    

    # Third convolution layer 
    tf.keras.layers.Conv2D(256, (3,3), activation='relu',use_bias=True),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),

    # Fourth convolution layer  
    tf.keras.layers.Conv2D(64, (3,3), activation='relu',use_bias=True),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),

    # Flatten the pooled feature maps
    tf.keras.layers.Flatten(),

    # Fully connected hidden layer
    tf.keras.layers.Dense(8, activation='relu',use_bias=True),

    # Output layer
    tf.keras.layers.Dense(1, activation='sigmoid',activity_regularizer=regularizers.L1(0.001))   

])
'''

"\nmodel = tf.keras.models.Sequential([\n                                    \n    # First convolution layer \n    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(64, 64, 3),use_bias=True),\n    tf.keras.layers.MaxPooling2D(2, 2),\n    tf.keras.layers.Dropout(0.25),\n\n    # Second convolution layer \n    tf.keras.layers.Conv2D(128, (3,3), activation='relu',use_bias=True),\n    tf.keras.layers.MaxPooling2D(2,2),\n    \n\n    # Third convolution layer \n    tf.keras.layers.Conv2D(256, (3,3), activation='relu',use_bias=True),\n    tf.keras.layers.MaxPooling2D(2,2),\n    tf.keras.layers.Dropout(0.25),\n\n    # Fourth convolution layer  \n    tf.keras.layers.Conv2D(64, (3,3), activation='relu',use_bias=True),\n    tf.keras.layers.MaxPooling2D(2,2),\n    tf.keras.layers.Dropout(0.25),\n\n    # Flatten the pooled feature maps\n    tf.keras.layers.Flatten(),\n\n    # Fully connected hidden layer\n    tf.keras.layers.Dense(8, activation='relu',use_bias=True),\n\n    # Output 

**Optimizer Implementation and model training/validation**

In [1]:
device_name = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)
model.to(device)

device

NameError: name 'torch' is not defined

In [19]:
target_acc = 99.6
epochs = 500

y_pred = []
y_true = []

tr_acc = []
valid_acc = []
tr_loss = []
valid_loss = []

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = nn.BCELoss()

for epoch in range(epochs):
    total_correct = 0
    total_samples = 0

    model.train()
    for images, labels in train_dl:
        optimizer.zero_grad()
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        
        predicted = (outputs >= 0.5).float()  # 0.5를 기준으로 클래스를 나눕니다
        
        correct = (predicted.squeeze(1) == labels).sum().item()
        total_correct += correct
        total_samples += labels.size(0)

        loss = loss_fn(outputs, labels.unsqueeze(1).float())
        loss.backward()
        optimizer.step()

    acc = 100 * total_correct / total_samples
    tr_acc.append(acc)
    tr_loss.append(loss.item())
    print(f'Epoch {epoch+1}: loss {loss: .4f}, Accuracy {acc: .2f}%')

    model.eval()
    with torch.no_grad():
        val_tot_correct = 0
        val_tot_samples = 0
        for images, labels in test_dl:
            images, labels = images.to(device), labels.to(device)
            val_outputs = model(images)

            val_pred = (val_outputs >= 0.5).float()  # 0.5를 기준으로 클래스를 나눕니다

            lab = labels.cpu().numpy()
            y_pred.extend(val_pred)
            y_true.extend(lab)
            
            val_cor = (val_pred.squeeze(1) == labels).sum().item()
            val_tot_correct += val_cor
            val_tot_samples += labels.size(0)

            val_loss = loss_fn(val_outputs, labels.unsqueeze(1).float())


        val_acc = 100 * val_tot_correct / val_tot_samples
        valid_acc.append(val_acc)
        valid_loss.append(val_loss.item())
        print(f'val_loss {val_loss: .4f}, Val_Accuracy {val_acc: .2f}%')

        if val_acc >= target_acc:
            print(f"Reached target accuracy of {val_acc: .4f}, stop training")
            break

Epoch 1: loss  0.0656 Accuracy  97.72%
Val_Accuracy  97.04%
Epoch 2: loss  0.0187 Accuracy  97.75%
Val_Accuracy  97.45%
Epoch 3: loss  0.0118 Accuracy  97.92%
Val_Accuracy  97.29%
Epoch 4: loss  0.0140 Accuracy  97.83%
Val_Accuracy  97.45%
Epoch 5: loss  0.0798 Accuracy  97.85%
Val_Accuracy  97.25%
Epoch 6: loss  0.0688 Accuracy  97.85%
Val_Accuracy  97.33%
Epoch 7: loss  0.0229 Accuracy  97.87%
Val_Accuracy  97.37%
Epoch 8: loss  0.1112 Accuracy  97.87%
Val_Accuracy  97.57%
Epoch 9: loss  0.2399 Accuracy  98.05%
Val_Accuracy  97.45%
Epoch 10: loss  0.0974 Accuracy  98.07%
Val_Accuracy  97.62%
Epoch 11: loss  0.0202 Accuracy  98.13%
Val_Accuracy  96.79%
Epoch 12: loss  0.0387 Accuracy  98.14%
Val_Accuracy  97.08%
Epoch 13: loss  0.0191 Accuracy  98.17%
Val_Accuracy  96.75%
Epoch 14: loss  0.0375 Accuracy  98.05%
Val_Accuracy  97.41%
Epoch 15: loss  0.0762 Accuracy  98.05%
Val_Accuracy  96.96%
Epoch 16: loss  0.0534 Accuracy  98.29%
Val_Accuracy  97.45%
Epoch 17: loss  0.0154 Accuracy  

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

label_mapping = {'ASMI': 0, 'Normal': 1}

y_pred_cpu = [pred.cpu().numpy() if isinstance(pred, torch.Tensor) else pred for pred in y_pred]
y_true_cpu = [true.cpu().numpy() if isinstance(true, torch.Tensor) else true for true in y_true]

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['ASMI', 'Normal'])
disp.plot()

In [ ]:
TP = cm[1, 1]
TN = cm[0, 0]
FP = cm[0, 1]
FN = cm[1, 0]

sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)
precision = TP / (TP + FP)

print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Precision: {precision:.4f}")

In [ ]:
plt.plot(range(1,epochs+1,1), tr_acc)
plt.plot(range(1,epochs+1,1), valid_acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

Training Accuracy:  0.9789062738418579
Validation Accuracy:  0.9906250238418579
Validation Specificity:  1.0
Validation Sensitivity:  1.0
Validation Recall:  1.0
Validation Precision:  0.9983713626861572
Validation Loss:  0.04193344712257385


In [ ]:
plt.plot(range(1,epochs+1,1), tr_loss)
plt.plot(range(1,epochs+1,1), valid_loss)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()